## Chapter 3

    - This notebook contains the code for Chapter 3 of the book.

In [3]:
from getpass import getpass
import os
OPENAI_API_KEY = getpass()

In [4]:
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

### Recipe 3.1.1a Wrapping RunnableLambda around a Python function

In [1]:
from langchain_core.runnables import RunnableLambda
import re

# Step 1: Define a preprocessing function to sanitize input
def sanitize_text(text):
    censored_list = ["shit", "damn", "hell"]
    for bad_word in censored_list:
        text = re.sub(rf"\b{bad_word}\b", "[censored]", text, flags = re.IGNORECASE)
    return text.strip()
 
# Step 2: Wrap this function in RunnableLambda
sanitize_text_runnable = RunnableLambda(sanitize_text)
 
# Stape 3: Invoke it
sanitize_text_runnable.invoke("What is this damn response?")


'What is this [censored] response?'

### Recipe 3.1.1b Building a chain with RunnableLambda and other components (OpenAI model)

In [5]:
from langchain_core.runnables import RunnableLambda
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
import re
 
## Step 1: Define a custom function
def sanitize_text(text):
    censored_list = ["shit", "damn", "hell"]
    for bad_word in censored_list:
        text = re.sub(rf"\b{bad_word}\b", "[censored]", text, flags = re.IGNORECASE)
    return text.strip()
 
## Step 2: Wrap it in RunnableLambda
sanitize_text_runnable = RunnableLambda(sanitize_text)
 
## Step 3: Define the template string for the ChatPromptTemplate
messages =[
    
    ("system","You are a helpful assistant for school administration."),
    
    ("human", "How much is the annual tuition fee for Grade 8?"),
    ("ai", "The annual tuition fee for Grade 8 is $4,200. You can pay in three installments."),
    ("human", "What is [censored]?"),
    ("ai", "Part of your question/statement has been censored question/statement, kindly rephrase without profanity"),
 
    ("human", "{user_question}")
] 
 
## Step 4: Link the message with the template
chat_template = ChatPromptTemplate.from_messages(
    messages=messages
)
 
## Step 5: Define the model and the llm chat
llm = ChatOpenAI(api_key=OPENAI_API_KEY, 
                 model = "gpt-4o-mini",
                 temperature = 0)
 
## Step 6: Define the chain
chain = {"user_question":sanitize_text_runnable} | chat_template | llm
 
## Step 7: Invoke the chain
response = chain.invoke("What do you know about the damn annual tuition fee?")
 
## Print the response
print(response.content)


It seems that part of your question has been censored. If you could provide more context or clarify what specific information you are looking for regarding the annual tuition fee, I would be happy to assist you!


### Recipe 3.1.2 Building a chain with RunnableSequence and other components (OpenAI model)

##### Steps 1 – 3:

In [10]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnableSequence
import re
 
## Step 1: Define a custom function for sanitization
def sanitize_text(text):
    censored_list = ["shit", "damn", "hell"]
    for bad_word in censored_list:
        text = re.sub(rf"\b{bad_word}", "[censored]", text, flags = re.IGNORECASE)
    return text.strip()
 
## Step 2: Define a custom function for validation of input
def validate_question(input: dict)->dict:
    q = input.get("user_question", " ").lower()
    school_keywords = ["fee", "holiday", "school"]
    if not any(keyword in q for keyword in school_keywords):
        raise ValueError("Sorry, I can only assist with school-related questions.")
    return {"validated_question": input["user_question"]}
 
## Step 3: Transform the custom functions into runnables
sanitize_text_runnable = RunnableLambda(sanitize_text)
validate_runnable = RunnableLambda(validate_question) 


##### Steps 4 – 5:

In [11]:
## Step 4: Define the template string for the ChatPromptTemplate
messages =[
    
    ("system","You are a helpful assistant for school administration."),
    
    ("human", "How much is the annual tuition fee for Grade 8?"),
    ("ai", "The annual tuition fee for Grade 8 is $4,200. You can pay in three installments."),
    ("human", "What is [censored]?"),
    ("ai", "Part of your question/statement has been censored question/statement, rephrase without profanity"),
 
    ("human", "{validated_question}")
]
 
## Step 5: Link the message with the template
chat_template = ChatPromptTemplate.from_messages(
    messages=messages
) 


##### Steps 6 – 9:

In [12]:
## Step 6: Define the model and the llm chat
llm = ChatOpenAI(api_key=OPENAI_API_KEY, 
                 model = "gpt-4o-mini",
                 temperature = 0,
                 max_tokens = 100)
 
 
## Step 7: Define the input list
question_list =[
    "Is there a holiday coming up that affects school schedules?",
    "What do you know about the damn annual tuition fee?",
    "Tell me more about the annual tuition fee."
    
]

 
## Step 8: Compose the chain using the RunnableSequence
full_chain = RunnableSequence({"user_question":sanitize_text_runnable},
                              {"validated_question":validate_runnable},
                              chat_template,
                              llm)
 
# Step 9: Call the .batch method
response = full_chain.batch(question_list)


In [13]:
print(response[0].content)
print("\n")
print(response[1].content)
print("\n")
print(response[2].content)


Yes, there are upcoming holidays that may affect school schedules. Please check the school calendar for specific dates and any related schedule changes. If you need information about a particular holiday, feel free to ask!


It seems that part of your question has been censored. If you are asking about the annual tuition fee for a specific program or school, please provide the name of the school or program, and I will do my best to assist you with the information you need.


The annual tuition fee for Grade 8 is $4,200. This fee typically covers various educational expenses, including classroom materials, access to facilities, and extracurricular activities. The tuition can usually be paid in three installments throughout the academic year. If you have any specific questions about what the tuition includes or payment options, feel free to ask!


### Recipe 3.1.3a – Setting up the LLM-as-judge with procedure

##### Steps 1-3

In [6]:
# Import necessary modules
from langchain_core.runnables import RunnableParallel, RunnableLambda, RunnableSequence
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
 
## Step 1:Define template string
## Take note of the variable we introduced here
messages = [
        ("system", "You are a helpful assistant for school administration."),
        ("human", "Explain in layman the topic: {topic}"),  
        ]

## Step 2a:Connect the template string to the chat prompt template
llm1_prompt = ChatPromptTemplate.from_messages(
    messages=messages
        
)

## Step 2b:Connect the template string to the chat prompt template
llm2_prompt = ChatPromptTemplate.from_messages(
    messages=messages
)
 
## Step 3:Define prompt for the judge model to compare LLM1 vs LLM2
judge_prompt = ChatPromptTemplate.from_messages(
    messages = [
        ("system", "You are to act as a judge to determine the simplicity of language between two responses."),
        ("human", """Don't re-define and don't add anything, just compare in terms 
        of simplicity: {llm1_response} or {llm2_response}?"""),
        ]
) 
 


##### Steps 4-7:

In [7]:
## Step 4:Initialize the LLMs and assign models
model1 = ChatOpenAI(api_key=OPENAI_API_KEY, model = "gpt-4o-mini", max_tokens = 90)
model2 = ChatOpenAI(api_key=OPENAI_API_KEY, model = "gpt-4.1-nano", max_tokens = 90)
judge_llm = ChatOpenAI(api_key=OPENAI_API_KEY, model = "gpt-4o-mini", max_tokens = 90)
 
## Step 5:Bind templates to models to create runnable LLM chains
llm1_chain = llm1_prompt| model1 
llm2_chain = llm2_prompt | model2
llm_judge_chain = judge_prompt | judge_llm
 
 
## Step 6:Identity passthrough to forward topic along the chain
identity_chain = RunnableLambda(lambda x: x['topic'])
 
## Step 7: Branch out input to run both LLMs and retain original topic
parallel_chain = RunnableParallel({
    "llm1": llm1_chain,
    "llm2": llm2_chain,
    "topic": identity_chain}
)
 


##### Steps 8-9

In [8]:
## Step 8: Let’s temporarily invoke the parallel chain to observe
response = parallel_chain.invoke({"topic": "What does the grading system mean?"}) 


## The output is a dictionary. So, we extract as:
print(response["llm1"].content)
print("\n")
print(response["llm2"].content)
print("\n")
print(response["topic"])


The grading system is a framework used by educational institutions to assess and communicate students' performance and understanding of course material. Typically, grades are assigned based on a combination of assignments, tests, participation, and overall engagement. Common grading scales include letter grades (A, B, C, etc.), numerical scores (0-100), or pass/fail options. These grades serve multiple purposes: they reflect a student's academic achievement, provide feedback for improvement, and


The grading system is a method used by educational institutions to evaluate and represent students' academic performance. It assigns letter grades, numbers, or descriptive levels to indicate how well a student has mastered the material. The system helps communicate achievement levels, determine eligibility for advancement or graduation, and provide feedback for improvement. Different institutions may use varying grading scales, such as A-F, percentage scores, or other symbols, but their prima

NOTE: 
    
    >
    
        Before moving to the next recipe, note that it is also possible to write the line for the RunnableParallel (in Step 8), not as a dictionary, but as:

In [12]:
from langchain_core.runnables import RunnablePassthrough
parallel_chain2 = RunnableParallel(llm1 = llm1_chain,
                                  llm2 = llm2_chain,
                                  topic = RunnablePassthrough()
)


## Step 8: We invoke the chain
response2 = parallel_chain2.invoke("What does the grading system mean?") 

## The output is a dictionary. So, we extract as:
print(response2["llm1"].content)
print("\n")
print(response2["llm2"].content)
print("\n")
print(response2["topic"])

The grading system is a standardized method used to evaluate and communicate a student's academic performance. It typically includes a range of letter grades (A, B, C, D, F) or numerical scores (0-100), with each grade reflecting a student's level of understanding and mastery of the material. 

Grades are often associated with specific criteria, such as assignments, exams, and participation, and they can influence student progression, eligibility for programs, and GPA


The grading system is a method used by educational institutions to evaluate and represent a student's academic performance. It typically assigns letter grades, percentages, or scores to indicate how well a student has mastered the material. These grades help communicate a student's progress, determine eligibility for advancement, and provide feedback for improvement. Different schools may have varying grading scales, but the overall purpose is to assess learning outcomes consistently.


What does the grading system mean

### Recipe 3.1.3b – Completing the LLM-as-judge procedure

NOTE:
    
    > The code in this section should be combined with the evaluations of Steps 1-9 above.

In [17]:
## Step 9: Evaluate and compare LLM1 vs LLM2 using a critic model
# The evaluate function processes the outputs coming out of the final parallel chain
def evaluate(parallel_output):
    topic = parallel_output["topic"]
    llm1_response = parallel_output["llm1"].content
    llm2_response = parallel_output["llm2"].content
    judge_list = [{"llm1_response":llm1_response, "llm2_response":llm2_response}]
    ## Call the .batch method to process 
    judgement = llm_judge_chain.batch(judge_list)[0].content        
    ### recall that .batch() method returns a list ()
    return {"llm1_response":llm1_response, 
            "llm2_response":llm2_response, 
            "evaluation": judgement}
 
## Step 10: Wrap the evaluator as a runnable
evaluation_chain = RunnableLambda(evaluate)
 
## Step 11: Compose the full pipeline: generate explanations 
full_chain = RunnableSequence(parallel_chain, evaluation_chain)
 
## Step 12: Run the full pipeline with a topic prompt
output = full_chain.invoke({"topic": "What does the grading system mean?"})


## Print the output
print(f"LLM 1 response: {output["llm1_response"]}")
print("\n")
print(f"LLM 2 response: {output["llm2_response"]}")
print("\n")
print(f"The verdict:{output["evaluation"]}")


LLM 1 response: The grading system is a method used to evaluate and communicate a student's academic performance and progress. It typically uses letters (A, B, C, D, F) or percentages to indicate levels of achievement in a subject or course. 

- **Letter Grades**: Commonly, an 'A' represents excellent performance, 'B' good performance, 'C' average, 'D' below average, and 'F' failing.
  
- **


LLM 2 response: The grading system is a method used by schools to evaluate and represent students' academic performance. It assigns letter grades or numerical scores based on how well students meet learning objectives, helping educators, students, and parents understand progress and areas needing improvement.


The verdict:The second response is simpler. It uses more straightforward language and avoids complex phrases, making it easier to understand. The first response contains more formal terminology and additional details that could make it less accessible.


### Recipe 3.1.4 – Building a book recommending pipeline using RunnablePassthrough and RunnableLambda

In [7]:
## Import necessary modules from Langchain
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
 
## Step 1: Define a function to get a list of recommended books
def get_recommended_books():
    return [
        "To Kill a Mockingbird by Harper Lee",
        "1984 by George Orwell",
        "Of Mice and Men by John Steinbeck",
        "Animal Farm by George Orwell"
    ]
 
## Step 2: Create a RunnableLambda to execute the get_recommended_books function
books_runnable = RunnableLambda(lambda _: get_recommended_books())
#books_runnable = RunnableLambda(get_recommended_books)
 
## Step 3: Define a prompt template to format the input for the LLM
prompt_template = PromptTemplate.from_template(
    template="Select {n} classic books recommended for secondary school students:\n{books}"
)
 
## Step 4: Initialize the ChatOpenAI model (replace with your actual API key)
llm = ChatOpenAI(
    model = "gpt-4o-mini",
    api_key=OPENAI_API_KEY
)
 
## Step 5: Create a runnable chain that passes the input 'n' 
## and the output of 'books_runnable' to the prompt, # then to the LLM
book_chain = {"n": RunnablePassthrough(), "books": books_runnable} | prompt_template | llm
 
## Step 6: Invoke the chain
output = book_chain.invoke({"n": 2})
print(output.content) 


Here are 2 classic books recommended for secondary school students:

1. **To Kill a Mockingbird by Harper Lee**
2. **1984 by George Orwell**


### Recipe 3.1.5 Dynamic routing with RunnableBranch 

##### Steps 1 - 3

In [8]:
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate
from langchain_core.runnables import RunnableLambda, RunnableBranch
from langchain_openai import ChatOpenAI
 
# Step 1: Define example question-answer pairs for admissions-related queries.
examples_qa_admissions =[
    {"question": "How do I update my contact information in my application", 
     "answer": "Log into your applicant portal, go to 'Personal Information' section to edit your details."},
    
    {"question": "My child is waitlisted. When will we know if they're accepted?", 
     "answer": "Waitlist decisions come by June 15th via email and portal."}
]
 
# Step 2: Define example question-answer pairs for exams-related queries.
examples_qa_exams =[
    {"question": "How can my child reschedule an exam due to a medical appointment?",
     "answer": "Kindly, submit the 'Exam Absence Request Form' from the school portal."},
    
    {"question": "Where can I find the midterm exam schedule?",
     "answer": "Access the schedule on the school website under 'Academic Calendar'."}
]
 


##### Steps 3 - 5

In [9]:
# Step 3: Create a prompt template for a single question-answer pair.
example_prompt = PromptTemplate(
    input_variables = ["question", "answer"],
    template = "Question: {question}\n Answer: {answer}"
)
 
# Step 4: Create a FewShotPromptTemplate for admissions questions, using the defined examples and prompt template above
few_shot_prompt_admissions = FewShotPromptTemplate(
    examples = examples_qa_admissions,
    example_prompt = example_prompt,
    input_variables=["user_question"],
    suffix="Q: {user_question}\nA:",
    prefix="""You are a helpful administrative assistant. 
    Below are examples of question-answer pairs for your guide.""",
)
 
# Step 5: Create a FewShotPromptTemplate for exams questions, using the defined examples and prompt.
few_shot_prompt_exams = FewShotPromptTemplate(
    examples = examples_qa_exams,
    example_prompt = example_prompt,
    input_variables=["user_question"],
    suffix="Q: {user_question}\nA:",
    prefix="""You are a helpful administrative assistant. 
    Below are examples of question-answer pairs for your guide.""",
)
 


##### Steps 6 – 8

In [10]:
# Step 6: Define the llm to be used.
llm = ChatOpenAI(
    model= "gpt-4o-mini",
    openai_api_key = OPENAI_API_KEY
)
 
# Step 7: Create the respective chain by combining the admissions prompt with the LLM.
admissions_chain = few_shot_prompt_admissions | llm
 
# Step 8: Create a chain by combining the exams prompt with the LLM.
exams_chain = few_shot_prompt_exams | llm


##### Steps 9 - 11

In [17]:
# Step 9: Define a function to route the user's question to the appropriate topic.
def route_topic(question: str):
    q = question.lower()
    if "admission" in q or "waitlist" in q or "application" in q:
        return "admissions"
    elif "exam" in q or "schedule" in q or "reschedule" in q:
        return "exams"

 
# Step 10: Define a fallback runnable to handle questions that don't match any specific topic.
default_runnable = RunnableLambda(lambda x: "I'm not sure how to answer that yet.")
 
# Step 11: Create a RunnableBranch to route the input based on the output of the route_topic function.
# The branch receives a dictionary as input
# Note: in the branch below, x represents the input dictionary containing user_question as a key
qa_router = RunnableBranch(
    (lambda x: route_topic(x["user_question"]) == "admissions", admissions_chain),
    (lambda x: route_topic(x["user_question"]) == "exams", exams_chain),
    default_runnable 
)


In [18]:
# Example usage
Q1 = qa_router.invoke({"user_question": "My child is waitlisted. When will we know if they're accepted?"})
#Q2 = qa_router.invoke({"user_question": "Where can I find the midterm exam schedule?"})
 
## Print the response
print(Q1.content)
#print(Q2.content)


Waitlist decisions are typically communicated by June 15th via email and on the portal.


### Recipe 3.2.1a String output parser with OpenAI model

In [20]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
 
## Step 1: Define a model
chat_model = ChatOpenAI(api_key=OPENAI_API_KEY, 
                        model = "gpt-4o-mini")
 
## Step 2: Define a template
template_string = PromptTemplate.from_template(
    inpput_variables = ["topic"],
    template = """
    Explain the topic provided below in 3 bullet points.
    Each bullet point should be concise and only be 1 sentence long.
    Topic: {topic}
    """
)
 
## Step 3: Establish a chain with no parser
chain_without_parser = template_string | chat_model
 
## Step 4: Establish a chain with parser
chain_with_parser = template_string | chat_model | StrOutputParser()

## Step 5: Invoking chain with no output parser
response_1 = chain_without_parser.invoke({"topic": "Learning objectives"})
print(response_1)
 


content='- Learning objectives clearly define the specific skills and knowledge that learners are expected to acquire by the end of an educational activity or program.  \n- They guide both instructors and students by providing a framework for lesson planning, assessment, and evaluation of progress.  \n- Well-crafted learning objectives are measurable, helping to ensure that educational outcomes can be effectively assessed.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 69, 'prompt_tokens': 42, 'total_tokens': 111, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': None, 'id': 'chatcmpl-BtsBOIvPRTVN6VmQaETN7zmQXd3yo', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--8b3cab74-621f-43e5-9287-c333f581f

In [21]:
## Step 6: Invoking chain with a string output parser
response_2 = chain_with_parser.invoke({"topic": "Learning objectives"})
print(response_2)

- Learning objectives clearly define what students are expected to know, understand, or be able to do by the end of a lesson or course.  
- They provide a framework for both instructors and learners, guiding the teaching process and assessing student progress.  
- Well-crafted learning objectives enhance engagement and motivation by making goals explicit and measurable.  


### Recipe 3.2.2 StructuredOutputParser() 

In [6]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain_openai import ChatOpenAI
 
## Step 1: Define the schema for the field
response_schema = [
    ## Field 1
    ResponseSchema(
        name = "topic",
        description= "The category of the question, such as admissions, exams, or fees"
),
   ## Field 1
    ResponseSchema(name="confidence", 
                   description="A  confidence score (from 0 to 1) reflecting certainty about the topic"
                   )
]

## Step 2: Link the schema to the parser
parser = StructuredOutputParser.from_response_schemas(response_schema)
 
print(parser.get_format_instructions())


The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"topic": string  // The category of the question, such as admissions, exams, or fees
	"confidence": string  // A  confidence score (from 0 to 1) reflecting certainty about the topic
}
```


In [7]:
from langchain_core.prompts import PromptTemplate
 
## Step 3: Define a prompt with format instructions as a partial variable
prompt = PromptTemplate.from_template(
    input_variable=["question"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
    template="""Classify the question and return:
    - topic (e.g., admissions, exams, fees)
    - confidence (a number between 0 and 1)
    {format_instructions}
    Question: {question}
    """
)
## Step 4: Define a model model
chat_model = ChatOpenAI(api_key=OPENAI_API_KEY, model="gpt-4o-mini")
 
## Step 5: Define a chain
full_chain = prompt | chat_model  | parser
 
## Step 6: Invoke the chain with a question
output = full_chain.invoke({"question": "I urgently need to finalize my child’s admission papers. Who can we speak to?"})
 
print(output) 
 



{'topic': 'admissions', 'confidence': 0.9}
